In [12]:
import MeCab
import pandas as pd
import numpy as np
import re
import csv

#def mecab_parse(file_in, file_out = file_in + ".mecab"): #NameError: name 'file_in' is not defined
def mecab_parse(file_in):
    
    file_out = file_in + ".mecab"

    tagger = MeCab.Tagger("-Ochasen")
    
    with open(file_in, encoding="utf-8") as input_file:
        with open(file_out, mode="w", encoding="utf-8") as output_file:
            parsed = tagger.parse(input_file.read())
            output_file.write(parsed)

    data = pd.read_table(file_out)
    data.columns = np.arange(0,data.shape[1])
    new_sorted_data = data[[0,1,3]].copy() #Only take Columns 0, 1, and 3

    new_sorted_data.columns = ["単語","ふりがな","品詞"]
    new_POS_series = new_sorted_data["品詞"]

    for index, POS in enumerate(new_POS_series):
        word = new_POS_series[index]
        if(type(word) == str):
            pos_of_hyphen = word.find("-")
            new_POS_series[index] = word[:pos_of_hyphen] #only get the prefix of part of speech (POS) because the original POS is too detailed

    new_sorted_data["品詞"] = new_POS_series

    new_sorted_data.loc[1, "品詞"]
    row_count = new_sorted_data.shape[0]

    functional_word_index = []
    for i in np.arange(row_count):
        pos_str =str(new_sorted_data.loc[i, "品詞"])
        word_str = str(new_sorted_data.loc[i, "単語"])
        
    #Remove words that are too common
        re_kanji = re.compile(r'^[\u4E00-\u9FD0]+$')
        re_roman = re.compile(r'^[a-zA-Z]+$') #a-z:小文字、A-Z:大文字
        re_katakana = re.compile(r'[\u30A1-\u30F4]+')
        re_hiragana = re.compile(r'^[あ-ん]+$')

        status_kanji = re_kanji.fullmatch(word_str)
        
        if pos_str in ["助詞", "記号"]:
            functional_word_index.append(i)
        elif status_kanji == None and len(word_str) < 5:
            functional_word_index.append(i)
    
    word_table = new_sorted_data.drop(functional_word_index)
    
    
    word_table = word_table.sort_values(by = "単語")
    word_table = word_table.drop_duplicates()
    word_table.index = np.arange(word_table.shape[0])
    return word_table.iloc[:,:]

In [13]:
result = mecab_parse("flu_detail.txt")
result

#result.to_csv("result2.csv" , encoding="utf-8") #default: encoding="utf-8"
#result.to_csv("result2.csv" , encoding="shift_jis")

vocab_list = result.iloc[:, 0]

list_of_words = []
count = 0
for i in vocab_list:
    list_of_words.append(i)

list_of_words

['Johnson',
 'Stevens',
 'じんましん',
 'アナフィラキシー',
 'アレルギー',
 'アレルギー性',
 'インフルエンザ',
 'ニューロパチー',
 'ネフローゼ',
 '一',
 '一方',
 '一般',
 '一過',
 '万',
 '下痢',
 '不',
 '両',
 '予診',
 '予防',
 '人',
 '今',
 '他',
 '以内',
 '低下',
 '体',
 '保健',
 '倦怠',
 '健康',
 '免疫',
 '入浴',
 '入院',
 '全身',
 '出',
 '出来',
 '分間',
 '判断',
 '前',
 '副',
 '効果',
 '動機',
 '化膿',
 '医師',
 '医療',
 '医薬品',
 '卵',
 '反応',
 '可能',
 '合併症',
 '吐',
 '呼吸',
 '嘱',
 '器',
 '困難',
 '場合',
 '多',
 '大量',
 '失神',
 '妊娠',
 '実施',
 '家族',
 '師',
 '庫',
 '当日',
 '形',
 '後',
 '徴',
 '心臓',
 '必要',
 '急',
 '急性',
 '性',
 '息',
 '悪寒',
 '意識',
 '感',
 '手足',
 '把握',
 '投与',
 '折',
 '指導',
 '指摘',
 '接種',
 '救済',
 '散在',
 '数',
 '斑',
 '方',
 '日',
 '旨',
 '期待',
 '末梢',
 '本人',
 '検査',
 '様',
 '様子',
 '機器',
 '機構',
 '機能',
 '機関',
 '次',
 '歩行',
 '歳',
 '死亡',
 '気',
 '気管支',
 '法',
 '法人',
 '注射',
 '注意',
 '浮腫',
 '消失',
 '清潔',
 '減少',
 '炎',
 '炎症',
 '熱',
 '熱性',
 '状態',
 '独立',
 '生活',
 '異常',
 '疾患',
 '疾病',
 '病',
 '病気',
 '症',
 '症候群',
 '症状',
 '痛',
 '発作',
 '発熱',
 '発疹',
 '発病',
 '発症',
 '発育',
 '白血球',
 '的',
 '皮膚',
 '相談',
 '眼',
 '破砕',
 '神経',
 '票',

In [ ]:
def get_jp_stress(vocab_list):
    import re
    import requests
    import time

    from bs4 import BeautifulSoup
    from random import random
    from selenium import webdriver

    driver = webdriver.Chrome()
    just_a_sec = random() 
    search_string = vocab_list
    
    stress_list =[]
    stress_list_with_word =[]
    for word in vocab_list:
        if len(stress_list) <= 300:
            #print(stress_list)
            try:    
                search_site = "https://www.weblio.jp/content/"
                search_site = search_site + word

                driver.get(search_site)
                time.sleep(just_a_sec * 1)

                search_box = driver.find_element_by_class_name("formBox")

                result_furigana = driver.find_element_by_xpath('//*[@id="cont"]/div[3]/div/div[1]/h2/b')
                result_stress = driver.find_element_by_xpath('//*[@id="cont"]/div[3]/div/div[1]/h2/span')

                furigana = result_furigana.text
                word_stress = result_stress.text
#                 if any(i.isdigit() for i in word_stress) == True:
#                     word_stress = word_stress.replace("[", "")
#                     word_stress = word_stress.replace("]", "")
                stress_list.append(word_stress)
                stress_list_with_word.append(word + ": " + furigana + ": " + word_stress)
            except:
                stress_list.append("[na]")
                stress_list_with_word.append(word + ": " + "[na]")
    
    return stress_list, stress_list_with_word
 
dic_result = get_jp_stress(list_of_words)
dic_result

In [2]:
dic_result
df = pd.DataFrame(dic_result[0], dic_result[1])

df.iloc[0:20,:]

NameError: name 'dic_result' is not defined

In [3]:
#import csv
# result.loc[:, 10] = dic_result[1]
# result

new_result = result.drop(["test","test2","test3", 1, 10], axis=1)
new_result.insert(2, "stress", dic_result[1])
new_result

new_result.to_csv("result3.csv" , encoding="shift_jis")
#new_result.to_csv("result3.csv" , encoding="utf8")

NameError: name 'result' is not defined

In [1]:
test

NameError: name 'test' is not defined